In [1]:
import pandas as pd
#from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
df = pd.read_csv(r"C:\Users\carlo\Repos\TrainingsTool\Tennis\Datasets\charting-m-points-from-2017-new.csv", encoding= 'unicode_escape', low_memory=False)

In [3]:
# only take matches where a specific player played
df = df.loc[df["match_id"].str.contains("Novak_Djokovic", case = True)]

In [4]:
# only keep rows where Djoko is serving
df = df[df.Serving == "ND"]

In [4]:
df = df.drop(["isAce", "Gm#", "match_id", "isUnret", "Svr", "Ret", "isRallyWinner", "isForced", "isUnforced", "isDouble", "Serving", "TBpt","Notes", "1stSV", "2ndSV", "1stIn", "2ndIn"], axis=1)

In [5]:
df = df[["Pt", "Set1", "Set2", "Gm1", "Gm2", "Pts", "TbSet", "TB?", "PtWinner", "isSvrWinner", "rallyCount", "1st", "2nd"]]

In [6]:
# seperate the Pts column to create pts_A and pts_B, splitting at the "-" seperator
df[["Pts1", "Pts2"]] = df.Pts.str.split("-", expand = True)

In [7]:
df = df.drop("Pts", axis=1)

In [8]:
df = df[["Pt", "Set1", "Set2", "Gm1", "Gm2", "Pts1", "Pts2", "TbSet", "TB?", "PtWinner", "isSvrWinner", "rallyCount", "1st", "2nd"]]

In [9]:
# Now we need to translate "AD" in Pts1 and Pts2 to "50"
columns = ["Pts1", "Pts2"]
for i in columns:
    df[str(i)] = df[str(i)].str.replace('AD', '50')

In [10]:
df = df.reset_index(drop=True)

In [11]:
test_df = df
test_df = test_df.astype({"1st": str, "2nd": str})

In [12]:
 # join the two ralley colums together into one column with "," seperator
test_df["1st&2nd"] = test_df[["1st", "2nd"]].apply(lambda x: "".join(x), axis=1)

In [13]:
test_df = test_df.drop(["1st", "2nd"], axis = 1)

In [14]:
ralley_df = test_df["1st&2nd"]

In [15]:
# creating a dataframe to store the shot_lists in
shot_list_dataframe = pd.DataFrame()

In [16]:
ralley_df_test = ralley_df.head(20)


In [18]:
ralley_df

0                    4d5n
1                   4#nan
2              4n5f29f3n#
3                6f3x#nan
4                   4#nan
               ...       
295348    4b1b1f1b1f1*nan
295349        6w5f1b2b3d@
295350               4w6d
295351       6s1f1f2n#nan
295352           6s2n#nan
Name: 1st&2nd, Length: 295353, dtype: object

In [19]:
possible_shots = ["f", "b", "r", "s", "v", "z", "o", "p", "y", "l", "m", "h", "i", "j", "k", "t", "u"]
serve_direction_second_serve = ["4", "5", "6"]
len_of_longest_ralley = 0
shot_list_dataframe = pd.DataFrame()

# i is a ralley
for i in ralley_df_test:
    # Initializing new shot for each new ralley
    shot = ""
    shot_list = []
    # every char in each ralley will be looked at
    for char in range(0, len(i)):
        # if the char is at position 3 in the ralley and its in serve direction list, its added to shot_list
        if i[char] in serve_direction_second_serve and char == 2:
            shot_list.append(shot)
            shot = ""
            shot = i[char]
             
        elif i[char] not in possible_shots and i[char] not in serve_direction_second_serve:
            shot = shot + str(i[char])
        # in all other cases, shot is done and appended to the shot_list
        else:
            shot_list.append(shot)
            shot = str(i[char])
    # the last shot has to be appended here, because it is not going into the loop again
    shot_list.append(shot)

    # if the last element of the shot list is "nan" we delete that because we dont want to predict "nan"
    if shot_list[-1][-1] == "n" and shot_list[-1][-2] == "a"  and shot_list[-1][-3] == "n" :
        shot_list[-1] = shot_list[-1].rstrip(shot_list[-1][-1])
        shot_list[-1] = shot_list[-1].rstrip(shot_list[-1][-1])
        shot_list[-1] = shot_list[-1].rstrip(shot_list[-1][-1])
        
    # finding the longest shot_list/ralley in the dataset
    if len(shot_list) > len_of_longest_ralley:
        len_of_longest_ralley = len(shot_list)

    shot_list_series = pd.Series(shot_list)
    shot_list_small_df = pd.DataFrame(shot_list)

    # now the shot list is beautiful and can add it to the empyt shot_list_dataframe
    shot_list_dataframe = pd.concat([shot_list_dataframe, shot_list_small_df], ignore_index=True, axis=1)

df = shot_list_dataframe

In [20]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,,,,,,,,,,,,,,,,,,,,
1,4d,4#,4n,6,4#,6w,4*,6d,4w,6n,4w,4n,4w,6,4n,6*,4n,6w,4*,4
2,5n,NaN,5,f3x#,NaN,4x,NaN,4d,6,4,6,4,5,r29,5n,NaN,6,4,NaN,b38
3,NaN,NaN,f29,NaN,NaN,NaN,NaN,NaN,b27,b39,b18,b38,b17,f18,NaN,NaN,b38,b3d#,NaN,b39
4,NaN,NaN,f3n#,NaN,NaN,NaN,NaN,NaN,f38,f19,f38,b28,f39,f1n@,NaN,NaN,b38,NaN,NaN,b38
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b27,f28,b38,f17,b38,NaN,NaN,NaN,b28,NaN,NaN,f38*
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f38,f38*,b39,f19,b38,NaN,NaN,NaN,f+18,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b38,NaN,b38,f29,s39,NaN,NaN,NaN,f17w#,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b39,NaN,f38,f1n@,f38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b38,NaN,s18,NaN,b39,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df = df.drop(0)
df = df.reset_index()
df = df.drop("index", axis = 1)

In [22]:
for i in range(0, len(ralley_df_test)):
    ralley = ralley_df_test[i]
    if ralley[-1] == "n" and ralley[-2] == "a" and ralley[-3] == "n":
        ralley_df_test[i] = ralley_df_test[i].rstrip(ralley_df_test[i][-1])
        ralley_df_test[i] = ralley_df_test[i].rstrip(ralley_df_test[i][-1])
        ralley_df_test[i] = ralley_df_test[i].rstrip(ralley_df_test[i][-1])

In [24]:
# df_each_char is created, each char is seperated into columns
shot_list_dataframe = pd.DataFrame()

# i is a ralley
for i in ralley_df_test:
    # Initializing new shot for each new ralley
    shot = ""
    shot_list = []
    # every char in each ralley will be looked at
    for char in range(0, len(i)):
        shot = i[char]
        shot_list.append(shot)
        shot = ""
        
    # the last shot has to be appended here, because it is not going into the loop again
    shot_list_series = pd.Series(shot_list)
    shot_list_small_df = pd.DataFrame(shot_list)

    # now the shot list is beautiful and can add it to the empyt shot_list_dataframe
    shot_list_dataframe = pd.concat([shot_list_dataframe, shot_list_small_df], ignore_index=True, axis=1)

df_each_char = shot_list_dataframe

In [26]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,4d,4#,4n,6,4#,6w,4*,6d,4w,6n,4w,4n,4w,6,4n,6*,4n,6w,4*,4
1,5n,NaN,5,f3x#,NaN,4x,NaN,4d,6,4,6,4,5,r29,5n,NaN,6,4,NaN,b38
2,NaN,NaN,f29,NaN,NaN,NaN,NaN,NaN,b27,b39,b18,b38,b17,f18,NaN,NaN,b38,b3d#,NaN,b39
3,NaN,NaN,f3n#,NaN,NaN,NaN,NaN,NaN,f38,f19,f38,b28,f39,f1n@,NaN,NaN,b38,NaN,NaN,b38
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b27,f28,b38,f17,b38,NaN,NaN,NaN,b28,NaN,NaN,f38*
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f38,f38*,b39,f19,b38,NaN,NaN,NaN,f+18,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b38,NaN,b38,f29,s39,NaN,NaN,NaN,f17w#,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b39,NaN,f38,f1n@,f38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b38,NaN,s18,NaN,b39,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b39,NaN,f39*,NaN,f38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
